# Install Adversarial robustness toolbox

In [67]:
pip install adversarial-robustness-toolbox


In [68]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch
import torchvision.models as models
from torchvision import transforms as T

from PIL import Image
from art.attacks.evasion import *
from art.estimators.classification import PyTorchClassifier

transform = T.Compose([            
 T.Resize(256),                    
 T.CenterCrop(224),                
 T.ToTensor(),                     
 T.Normalize(mean=[0.485, 0.456, 0.406],
             std=[0.229, 0.224, 0.225])
 ])

In [ ]:
model = models.resnet18(pretrained=True).eval()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

classifier = PyTorchClassifier(
    model=model,
    clip_values=(0, 1),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 224, 224),
    nb_classes=1000,
)

In [ ]:
# url, filename = ("http://www.cs.cornell.edu/courses/cs5670/2019sp/projects/pa5/assignment_data/imagenet_classes.txt", "imagenet_classes.txt")
# try: urllib.URLopener().retrieve(url, filename)
# except: urllib.request.urlretrieve(url, filename)

with open('imagenet_classes.txt') as f:
  classes = [line.strip() for line in f.readlines()]

#Upload Image

In [69]:
import urllib
url, filename = ("https://www.insidehook.com/wp-content/uploads/2020/04/JetBlue-Airplane.jpg?fit=1500%2C1000", "example.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

img = Image.open('airplane2.jpg')
transformed_img = transform(img)
input_img = transformed_img.unsqueeze(0)

#Generate and save Adversarial samples (Choose filename)

In [200]:
"""
e = 0.1
for a in range(5):
  attack = DeepFool(classifier, epsilon=e)
  advx = attack.generate(x=input_img)
  e = e+a
  rescaled = (advx - advx.min()) / (advx.max() - advx.min())
  rescaled = (255*np.transpose(np.squeeze(rescaled))).astype(np.uint8)
  rescaled = np.moveaxis(rescaled ,0, 1)
  output = Image.fromarray(rescaled)
  s = 'XXXXX_DeepFoolPlane' + str(e) + '.png'
  output.save(s, 'png')
"""

"\ne = 0.1\nfor a in range(5):\n  attack = DeepFool(classifier, epsilon=e)\n  advx = attack.generate(x=input_img)\n  e = e+a\n  rescaled = (advx - advx.min()) / (advx.max() - advx.min())\n  rescaled = (255*np.transpose(np.squeeze(rescaled))).astype(np.uint8)\n  rescaled = np.moveaxis(rescaled ,0, 1)\n  output = Image.fromarray(rescaled)\n  s = 'XXXXX_DeepFoolPlane' + str(e) + '.png'\n  output.save(s, 'png')\n"

#Load image to test

In [72]:
def loadimage(s):
  img = Image.open(s)
  transformed_img = transform(img)
  final_img = transformed_img.unsqueeze(0)
  return final_img

In [192]:
name = "name-of-image-file"
x = loadimage(name)
x.shape

torch.Size([1, 3, 224, 224])

#Select model for testing

In [193]:
modell = models.resnet18(pretrained=True).eval()
output = model(x)

#Output

In [194]:
_, index = torch.max(output, 1)
percentage = torch.nn.functional.softmax(output, dim=1)[0] * 100
print(classes[index[0]], percentage[index[0]].item())

spotlight, spot 46.485382080078125


In [195]:
_, indices = torch.sort(output, descending=True)
[(classes[idx], percentage[idx].item()) for idx in indices[0][:5]]

[('spotlight, spot', 46.485382080078125),
 ('bubble', 12.573700904846191),
 ('jellyfish', 8.003231048583984),
 ('traffic light, traffic signal, stoplight', 5.805872917175293),
 ('matchstick', 5.370112895965576)]